In [ ]:
import polars as pl
from benchmark_utils import mock_snakemake

DEBUG = False

if "snakemake" not in globals() or hasattr(snakemake, "mock"):  # noqa: F821
    snakemake = mock_snakemake("process_load_data")
    DEBUG = True

In [2]:
# read dataframe
df = pl.scan_csv(snakemake.input[0], has_header=False)
df.head().collect()

column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,column_36,column_37,…,column_8724,column_8725,column_8726,column_8727,column_8728,column_8729,column_8730,column_8731,column_8732,column_8733,column_8734,column_8735,column_8736,column_8737,column_8738,column_8739,column_8740,column_8741,column_8742,column_8743,column_8744,column_8745,column_8746,column_8747,column_8748,column_8749,column_8750,column_8751,column_8752,column_8753,column_8754,column_8755,column_8756,column_8757,column_8758,column_8759,column_8760
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""26.1354+7.4724i""","""25.2687+7.1414i""","""24.5124+6.8526i""","""24.0601+6.6799i""","""23.8858+6.6133i""","""24.2236+6.7423i""","""24.9537+7.0211i""","""25.4077+7.1945i""","""24.9935+7.0363i""","""24.1981+6.7326i""","""23.5666+6.4914i""","""22.8949+6.2349i""","""22.2662+5.9948i""","""22.0924+5.9284i""","""22.4746+6.0744i""","""23.4559+6.4491i""","""25.3607+7.1765i""","""28.9068+8.5307i""","""31.151+9.3878i""","""31.0637+9.3544i""","""30.813+9.2587i""","""30.0811+8.9792i""","""28.6121+8.4182i""","""26.8346+7.7394i""","""25.4202+7.1469i""","""24.5794+6.8342i""","""24.0069+6.6193i""","""23.923+6.5762i""","""24.3761+6.7218i""","""25.7729+7.1867i""","""27.8862+7.8795i""","""30.0521+8.5988i""","""30.3499+8.6509i""","""29.4338+8.2739i""","""28.2976+7.8217i""","""27.396+7.4821i""","""26.6115+7.1945i""",…,"""28.1287+7.7937i""","""27.6338+7.6378i""","""27.2477+7.5193i""","""27.1995+7.541i""","""27.7932+7.8173i""","""29.0565+8.3258i""","""31.9131+9.4068i""","""33.3311+9.9929i""","""32.987+9.8923i""","""32.3804+9.6801i""","""31.416+9.3413i""","""29.8308+8.7729i""","""27.7829+8.0126i""","""26.079+7.3801i""","""25.043+6.9876i""","""24.3317+6.7148i""","""24.0081+6.591i""","""24.0979+6.6165i""","""24.9785+6.9374i""","""26.628+7.5439i""","""28.0836+8.0767i""","""28.2653+8.1188i""","""27.3513+7.7513i""","""26.7258+7.5054i""","""26.2857+7.3419i""","""25.5732+7.0806i""","""25.4794+7.0542i""","""25.8306+7.2014i""","""26.0994+7.3202i""","""27.2923+7.7842i""","""30.1321+8.8655i""","""31.5043+9.404i""","""30.6455+9.0861i""","""29.6073+8.696i""","""28.6051+8.3228i""","""27.6322+7.9633i""","""26.8059+7.6548i"""
"""6.247+1.2666i""","""6.2149+1.2524i""","""6.1868+1.2399i""","""6.1701+1.2325i""","""6.1636+1.2296i""","""6.1761+1.2351i""","""6.2032+1.2472i""","""6.22+1.2546i""","""6.2047+1.2478i""","""6.1752+1.2347i""","""6.1517+1.2243i""","""6.1268+1.2133i""","""6.1035+1.2029i""","""6.0971+1.2i""","""6.1112+1.2063i""","""6.1476+1.2225i""","""6.2183+1.2539i""","""6.3498+1.3123i""","""6.4331+1.3492i""","""6.4298+1.3478i""","""6.4205+1.3437i""","""6.3934+1.3316i""","""6.3389+1.3074i""","""6.273+1.2781i""","""6.3307+1.2723i""","""6.2817+1.2556i""","""6.2526+1.245i""","""6.273+1.2473i""","""6.3474+1.2639i""","""6.5434+1.3098i""","""6.8622+1.3826i""","""7.1693+1.4542i""","""7.3101+1.4797i""","""7.3333+1.4737i""","""7.3295+1.461i""","""7.2863+1.4446i""","""7.2318+1.4277i""",…,"""7.2466+1.4461i""","""7.1586+1.4269i""","""7.0833+1.4109i""","""6.9971+1.3968i""","""6.9148+1.39i""","""6.9069+1.4021i""","""7.0336+1.4524i""","""6.9925+1.461i""","""6.915+1.445i""","""6.8514+1.4285i""","""6.7537+1.4028i""","""6.6171+1.3644i""","""6.4953+1.3234i""","""6.3939+1.2893i""","""6.3487+1.2712i""","""6.3247+1.2598i""","""6.3132+1.2546i""","""6.3351+1.259i""","""6.4003+1.2787i""","""6.5106+1.3136i""","""6.6131+1.3453i""","""6.6774+1.3574i""","""6.6823+1.3485i""","""6.6738+1.3405i""","""6.6479+1

In [ ]:
# unpivot
df2 = (
    df.with_row_index(name="bus", offset=1)
    .unpivot(index="bus", value_name="load", variable_name="hour")
    .with_columns(pl.col("hour").str.strip_prefix("column_").cast(pl.Int32))
    .sort("bus", "hour")
    # cache result since .unpivot cannot be streamed yet https://github.com/pola-rs/polars/issues/20947
    .collect()
    .lazy()
)
df2.head().collect()

bus,hour,load
u32,i32,str
0,1,"""26.1354+7.4724i"""
0,2,"""25.2687+7.1414i"""
0,3,"""24.5124+6.8526i"""
0,4,"""24.0601+6.6799i"""
0,5,"""23.8858+6.6133i"""


In [4]:
if DEBUG:
    issues = df2.filter(~pl.col("load").str.ends_with("i")).collect()
    if not issues.is_empty():
        raise ValueError(f"Some loads don't end with 'i' as expected\n: {issues}")

In [5]:
# keep only active loads
df3 = df2.with_columns(
    pl.col("load")
    .str.split_exact("+", 1)
    .struct.field("field_0")
    .cast(pl.Float64)
    .alias("active_load")
).drop("load")

df3.head().collect()

bus,hour,active_load
u32,i32,f64
0,1,26.1354
0,2,25.2687
0,3,24.5124
0,4,24.0601
0,5,23.8858


In [6]:
# parse date
df4 = df3.with_columns(
    (pl.col("hour") - 1) * pl.duration(hours=1) + pl.datetime(2019, 1, 1)
).rename({"hour": "datetime"})
df4.head(25).collect()

bus,datetime,active_load
u32,datetime[μs],f64
0,2019-01-01 00:00:00,26.1354
0,2019-01-01 01:00:00,25.2687
0,2019-01-01 02:00:00,24.5124
0,2019-01-01 03:00:00,24.0601
0,2019-01-01 04:00:00,23.8858
…,…,…
0,2019-01-01 20:00:00,30.813
0,2019-01-01 21:00:00,30.0811
0,2019-01-01 22:00:00,28.6121


In [7]:
# Confirm loads are always positive (i.e. dataset doesn't include generation) and drop zero loads
if DEBUG:
    assert df4.filter(pl.col("active_load") < 0).collect().height == 0, (
        "Negative active loads found!"
    )

df5 = df4.filter(pl.col("active_load") != 0)

print(
    df5.explain(optimized=True)
)  # Check the query is optimal (i.e. filter is done before other operations)

if DEBUG:
    df5 = df5.collect().lazy()  # cache to avoid double computation
    print(f"{df5.collect().height / df2.collect().height:.2%} of loads are non-zero.")

SELECT [col("bus"), col("hour").alias("datetime"), col("active_load")]
   WITH_COLUMNS:
   [[([([(col("hour")) - (1)]) * (1h)]) + (2019-01-01 00:00:00)]] 
    simple π 3/3 ["bus", "hour", "active_load"]
      FILTER [(col("active_load")) != (0.0)]
      FROM
         WITH_COLUMNS:
         [col("load").str.split_exact(["+"]).struct.field_by_name(field_0)().strict_cast(Float64).alias("active_load")] 
          DF ["bus", "hour", "load"]; PROJECT["bus", "hour", "load"] 3/3 COLUMNS
27.87% of loads are non-zero.


In [8]:
plt = None
if DEBUG:
    load_ca = df5.group_by("datetime").sum().drop("bus").collect()
    plt = (
        load_ca.group_by(pl.col("datetime").dt.hour())
        .mean()
        .plot.line(x="datetime", y="active_load")
        .properties(title="Average Load by Hour in California")
    )
plt

alt.Chart(...)

In [9]:
if DEBUG:
    plt = (
        load_ca.group_by(pl.col("datetime").dt.month())
        .mean()
        .plot.line(x="datetime", y="active_load")
        .properties(title="Average Load by Month in California")
    )
plt

alt.Chart(...)

In [10]:
# Save processed data
df5.sink_parquet(snakemake.output[0])